# UrbanSounds8k Data Processing

## Projenin İşleyişi

Öncelikle bize verilen bilgileri almak için not defterini okuyacağım. Bu not defterinde verilerin hangi kriterlere göre
adlandırıldığı belirtilmekte bu kriterlerle beraber sesleri ayırıp farklı klasörlere yerleştireceğim. Böylece projemizin ilk adımı olan veriyi işleme kısmının ilk adımını atmış olacağız. Ardından bu verileri bulundukları klasörde spektogramlara çevireceğim ve takımımızın model oluşturan kısmı için verileri algoritmaya sunulabilir hale getireceğim.

In [2]:

import pandas as pd
import numpy as np
import os
import librosa
import librosa.display
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import shutil
import csv
import cv2

In [3]:

with open("UrbanSound8K_README.txt", "r",encoding = "UTF-8") as file:
    readme = file.read()
    print(readme)

UrbanSound8K

Created By
----------

Justin Salamon*^, Christopher Jacoby* and Juan Pablo Bello*
* Music and Audio Research Lab (MARL), New York University, USA
^ Center for Urban Science and Progress (CUSP), New York University, USA
http://serv.cusp.nyu.edu/projects/urbansounddataset
http://marl.smusic.nyu.edu/
http://cusp.nyu.edu/

Version 1.0


Description
-----------

This dataset contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: air_conditioner, car_horn, 
children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer, siren, and street_music. The classes are 
drawn from the urban sound taxonomy described in the following article, which also includes a detailed description of 
the dataset and how it was compiled:

J. Salamon, C. Jacoby and J. P. Bello, "A Dataset and Taxonomy for Urban Sound Research", 
22nd ACM International Conference on Multimedia, Orlando USA, Nov. 2014.

All excerpts are taken from field recordings uploaded to www.freesou

## ClassID Meanings
[fsID]_[classID]_[occurrenceID]_[sliceID].wav şeklinde adlandırılmış bu dosyalarda class ID'lerini kullanıp veriler üzerinde 
ayırma yapabiliriz.

A numeric identifier of the sound class:

0 = air_conditioner

1 = car_horn

2 = children_playing

3 = dog_bark

4 = drilling

5 = engine_idling

6 = gun_shot

7 = jackhammer

8 = siren

9 = street_music


In [4]:
#İlk olarak fold1 üzerinde çalışacağız bunun için içindeki tüm dosyaların adlarını almak istiyorum. Bunun için ise os.listdir()
#komutunu kullanacağım

path = "C:/Users/FIRAT/Desktop/UrbanSound8K/audio/fold1"


filenames = os.listdir(path)
print(filenames)

['.DS_Store', '101415-3-0-2.wav', '101415-3-0-3.wav', '101415-3-0-8.wav', '102106-3-0-0.wav', '102305-6-0-0.wav', '102842-3-0-1.wav', '102842-3-1-0.wav', '102842-3-1-5.wav', '102842-3-1-6.wav', '103074-7-0-0.wav', '103074-7-0-1.wav', '103074-7-0-2.wav', '103074-7-1-0.wav', '103074-7-1-1.wav', '103074-7-1-2.wav', '103074-7-1-3.wav', '103074-7-1-4.wav', '103074-7-1-5.wav', '103074-7-1-6.wav', '103074-7-2-0.wav', '103074-7-3-0.wav', '103074-7-3-1.wav', '103074-7-3-2.wav', '103074-7-3-3.wav', '103074-7-4-0.wav', '103074-7-4-1.wav', '103074-7-4-2.wav', '103074-7-4-3.wav', '103074-7-4-4.wav', '103074-7-4-5.wav', '103074-7-4-6.wav', '103258-5-0-0.wav', '103258-5-0-1.wav', '103258-5-0-10.wav', '103258-5-0-11.wav', '103258-5-0-12.wav', '103258-5-0-13.wav', '103258-5-0-14.wav', '103258-5-0-15.wav', '103258-5-0-16.wav', '103258-5-0-17.wav', '103258-5-0-18.wav', '103258-5-0-19.wav', '103258-5-0-2.wav', '103258-5-0-3.wav', '103258-5-0-4.wav', '103258-5-0-5.wav', '103258-5-0-6.wav', '103258-5-0-7.wa

Artık elimizde fold 1 için tüm dosya isimleri var. Bunları indexleyerek ne sesi olduklarına ulaşabiliriz. Birinci "-" işaretinden hemen sonra gelen sayı bizim bunları sınıflandırmamız için yeterli olacaktır. Oldukları ana klasörün içine ilgili sesleri depolamak için kategorize edilmiş klasörler açacağım. Elimizdeki dosyaların tümünü python yardımı ile bunların içine kategorize edilmiş bir biçimde aktaracağım.

In [5]:
#klasörleri açmak için 
directory_names = ["air_conditioner", "car_horn","children_playing", "dog_bark", "drilling", "engine_idling", "gun_shot", "jackhammer",
                  "siren", "street_music"
                  ]
#ilk olarak processed data klasörünü oluşturalım
directory = "Processed_Data"
parent_dir = "C:/Users/FIRAT/Desktop/UrbanSound8K/"
path = os.path.join(parent_dir, directory)
os.mkdir(path)
print(f"{directory} klasörü oluşturuldu")
for i in directory_names:
    directory = i
    parent_dir = "C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data"
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    print(f"{directory} klasörü oluşturuldu")


Processed_Data klasörü oluşturuldu
air_conditioner klasörü oluşturuldu
car_horn klasörü oluşturuldu
children_playing klasörü oluşturuldu
dog_bark klasörü oluşturuldu
drilling klasörü oluşturuldu
engine_idling klasörü oluşturuldu
gun_shot klasörü oluşturuldu
jackhammer klasörü oluşturuldu
siren klasörü oluşturuldu
street_music klasörü oluşturuldu


In [6]:
#ardından ise elimizdeki fold1 klasörünü inceleyip buradaki koşullara göre veriyi doğru klasörlere aktaralım. Aktarmak için gerekli
#modülü dahil ediyorum ancak öncelikle bir sözlük yapısı oluşturup stabil bir kod yazacağım

import shutil
path = "C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data"
folders = os.listdir(path)
indexes = ["0","1","2","3","4","5","6","7","8","9"]
folder_dict = dict(zip(indexes, folders))
for i in filenames:
    if i.startswith(".DS"):
        continue
    else:
        
        isplitted = i.split("-")
        source = f"C:/Users/FIRAT/Desktop/UrbanSound8K/audio/fold1/{i}"
        destination = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{folder_dict[isplitted[1]]}"
        shutil.move(source, destination)
        print(f"{i} dosyası başarıyla {destination} klasörüne taşındı")

101415-3-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
101415-3-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
101415-3-0-8.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102106-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102305-6-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
102842-3-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102842-3-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102842-3-1-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
102842-3-1-6.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark k

164053-8-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
164053-8-2-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
164053-8-2-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
164053-8-2-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
164053-8-2-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
164053-8-3-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
164312-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
164312-3-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
164312-3-3-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
1

40722-8-0-7.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
40722-8-0-8.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
43786-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
43786-3-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
43806-8-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren klasörüne taşındı
44736-5-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
44736-5-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
46654-6-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
46656-6-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne

Birinci klasördeki verilerin hepsini başarıyla sınıflandırdık. Artık sıra geldi tüm diğer verileri sınıflandırmaya. Bunun için geri kalan klasörlerin hepsinde bir for döngüsü ile gezineceğim ve dosya isimlerini otomatize edip alacağım böylece daha az kod ile daha fazla iş başarabileceğiz.

In [7]:
path = "C:/Users/FIRAT/Desktop/UrbanSound8K/audio/"
unprocessed_file_names = os.listdir(path)
#fold1 klasörünü çıkardım çünkü onu zaten işlemiştik. Şimdi elimizde for döngüsünde kullanabileceğimiz bir liste var
unprocessed_file_names.remove(".DS_Store")
unprocessed_file_names.remove("fold1")
print(unprocessed_file_names)

['fold10', 'fold2', 'fold3', 'fold4', 'fold5', 'fold6', 'fold7', 'fold8', 'fold9']


In [8]:
path = "C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data"
folders = os.listdir(path)
indexes = ["0","1","2","3","4","5","6","7","8","9"]
folder_dict = dict(zip(indexes, folders))
for i in unprocessed_file_names:
    datapath = f"C:/Users/FIRAT/Desktop/UrbanSound8K/audio/{i}"
    filenames2 = os.listdir(datapath)
    
    for j in filenames2:
        if j.startswith(".DS"):
            continue
        else:
            
            jsplitted = j.split("-")
           
            source = f"C:/Users/FIRAT/Desktop/UrbanSound8K/audio/{i}/{j}"
            destination = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{folder_dict[jsplitted[1]]}"
            shutil.move(source, destination)
            print(f"{j} dosyası başarıyla {destination} klasörüne taşındı")
            

100648-1-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100648-1-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100648-1-2-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100648-1-3-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100648-1-4-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
100795-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
100795-3-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
100795-3-1-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
100795-3-1-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark k

167464-0-0-20.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-21.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-22.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-23.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-24.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
167464-0-0-6.wav dosyası başarıyla 

72538-3-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72538-3-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
73524-0-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
73524-0-0-101.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
73524-0-0-105.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
73524-0-0-113.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
73524-0-0-114.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
73524-0-0-126.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
73524-0-0-127.wav dosyası başarıyla C:/Users/FIRAT/D

147672-3-2-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
14780-9-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music klasörüne taşındı
14780-9-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music klasörüne taşındı
14780-9-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music klasörüne taşındı
147926-0-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
147926-0-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
147926-0-0-12.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
147926-0-0-16.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
147926-0-0-17.wav dosyası başarıyla C:/Users/FIRAT/Desk

201652-5-4-7.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
201652-5-4-8.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
201652-5-4-9.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
201652-5-5-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
201652-5-5-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
201652-5-5-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
201652-5-5-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
201652-5-5-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling klasörüne taşındı
201652-5-5-5.wav dosyası başarıyla C:/Users/FIRAT/Deskto

...

177726-0-0-31.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
177726-0-0-32.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
177726-0-0-33.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
177726-0-0-34.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
177726-0-0-36.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
177726-0-0-37.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
177726-0-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
177726-0-0-6.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
177726-0-0-7.wav dosyası başarıyla

71309-1-0-8.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
71309-1-1-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
71309-1-2-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn klasörüne taşındı
71700-3-3-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
71700-3-4-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
71700-3-4-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
71700-3-4-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
72015-2-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_playing klasörüne taşındı
72015-2-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_pl

105029-7-4-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer klasörüne taşındı
105088-3-0-10.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
105088-3-0-11.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
105088-3-0-19.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
105088-3-0-8.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
106955-6-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot klasörüne taşındı
110389-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
116483-3-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
116483-3-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_b

184449-2-0-33.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_playing klasörüne taşındı
184575-3-0-0.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
184575-3-0-1.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
184575-3-0-2.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
184575-3-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark klasörüne taşındı
185374-9-0-16.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music klasörüne taşındı
185374-9-0-18.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music klasörüne taşındı
185374-9-0-23.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music klasörüne taşındı
185374-9-0-30.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/

75743-0-0-21.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
75743-0-0-22.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
75743-0-0-23.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
75743-0-0-3.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
75743-0-0-4.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
75743-0-0-5.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
75743-0-0-6.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
75743-0-0-7.wav dosyası başarıyla C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner klasörüne taşındı
75743-0-0-8.wav dosyası başarıyla C:/Users/FI

In [9]:
list_dir_processed = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data")
print(list_dir_processed)

['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']


In [47]:
#Şimdi bu wav dosyalarını spektrogram şeklinde kaydetmek için gerekli kodlarımızı yazalım

def create_spectrogram(y):
    spec = librosa.feature.melspectrogram(y=y)
    spec_conv = librosa.amplitude_to_db(spec, ref =np.max)
    return spec_conv
for i in list_dir_processed:
    path = f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/"
    filenames = os.listdir(path)
    directory = "Spectrograms"
    folderpath = os.path.join(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/{i}/", directory)
    os.mkdir(folderpath)
    print(f"{directory} klasörü {path} konumunda oluşturuldu oluşturuldu")
    
    
    
    for j in filenames:
        path_of_file = path + f"/{j}"
        y, sr = librosa.load(path_of_file)
        spectrogram = create_spectrogram(y)
        fig = plt.Figure(figsize = (3,3), dpi=16)
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        ax.axis("off")
        p = librosa.display.specshow(spectrogram, ax=ax, y_axis='log', x_axis='time')
        new_filename = j.replace(".wav",".png")
        fig.savefig(f"{path}/Spectrograms/{new_filename}")
        
        
        print(f"Spectrogram başarıyla kaydedildi {new_filename}")
        
    
        
    


Spectrograms klasörü C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/ konumunda oluşturuldu oluşturuldu
Spectrogram başarıyla kaydedildi 100852-0-0-0.png
Spectrogram başarıyla kaydedildi 100852-0-0-1.png
Spectrogram başarıyla kaydedildi 100852-0-0-10.png
Spectrogram başarıyla kaydedildi 100852-0-0-11.png
Spectrogram başarıyla kaydedildi 100852-0-0-12.png
Spectrogram başarıyla kaydedildi 100852-0-0-13.png
Spectrogram başarıyla kaydedildi 100852-0-0-14.png
Spectrogram başarıyla kaydedildi 100852-0-0-15.png
Spectrogram başarıyla kaydedildi 100852-0-0-16.png
Spectrogram başarıyla kaydedildi 100852-0-0-17.png
Spectrogram başarıyla kaydedildi 100852-0-0-18.png
Spectrogram başarıyla kaydedildi 100852-0-0-19.png
Spectrogram başarıyla kaydedildi 100852-0-0-2.png
Spectrogram başarıyla kaydedildi 100852-0-0-20.png
Spectrogram başarıyla kaydedildi 100852-0-0-21.png
Spectrogram başarıyla kaydedildi 100852-0-0-22.png
Spectrogram başarıyla kaydedildi 100852-0-0-23.png
Spectrogram b

Spectrogram başarıyla kaydedildi 146690-0-0-87.png
Spectrogram başarıyla kaydedildi 146690-0-0-93.png
Spectrogram başarıyla kaydedildi 146709-0-0-1.png
Spectrogram başarıyla kaydedildi 146709-0-0-11.png
Spectrogram başarıyla kaydedildi 146709-0-0-12.png
Spectrogram başarıyla kaydedildi 146709-0-0-17.png
Spectrogram başarıyla kaydedildi 146709-0-0-18.png
Spectrogram başarıyla kaydedildi 146709-0-0-19.png
Spectrogram başarıyla kaydedildi 146709-0-0-20.png
Spectrogram başarıyla kaydedildi 146709-0-0-22.png
Spectrogram başarıyla kaydedildi 146709-0-0-25.png
Spectrogram başarıyla kaydedildi 146709-0-0-26.png
Spectrogram başarıyla kaydedildi 146709-0-0-3.png
Spectrogram başarıyla kaydedildi 146709-0-0-30.png
Spectrogram başarıyla kaydedildi 146709-0-0-33.png
Spectrogram başarıyla kaydedildi 146709-0-0-34.png
Spectrogram başarıyla kaydedildi 146709-0-0-37.png
Spectrogram başarıyla kaydedildi 146709-0-0-44.png
Spectrogram başarıyla kaydedildi 146709-0-0-48.png
Spectrogram başarıyla kaydedildi 

Spectrogram başarıyla kaydedildi 165454-0-0-8.png
Spectrogram başarıyla kaydedildi 165454-0-0-9.png
Spectrogram başarıyla kaydedildi 166942-0-0-0.png
Spectrogram başarıyla kaydedildi 166942-0-0-1.png
Spectrogram başarıyla kaydedildi 166942-0-0-10.png
Spectrogram başarıyla kaydedildi 166942-0-0-11.png
Spectrogram başarıyla kaydedildi 166942-0-0-12.png
Spectrogram başarıyla kaydedildi 166942-0-0-13.png
Spectrogram başarıyla kaydedildi 166942-0-0-14.png
Spectrogram başarıyla kaydedildi 166942-0-0-15.png
Spectrogram başarıyla kaydedildi 166942-0-0-2.png
Spectrogram başarıyla kaydedildi 166942-0-0-3.png
Spectrogram başarıyla kaydedildi 166942-0-0-4.png
Spectrogram başarıyla kaydedildi 166942-0-0-5.png
Spectrogram başarıyla kaydedildi 166942-0-0-6.png
Spectrogram başarıyla kaydedildi 166942-0-0-7.png
Spectrogram başarıyla kaydedildi 166942-0-0-8.png
Spectrogram başarıyla kaydedildi 166942-0-0-9.png
Spectrogram başarıyla kaydedildi 167464-0-0-0.png
Spectrogram başarıyla kaydedildi 167464-0-0-

Spectrogram başarıyla kaydedildi 178686-0-0-14.png
Spectrogram başarıyla kaydedildi 178686-0-0-15.png
Spectrogram başarıyla kaydedildi 178686-0-0-16.png
Spectrogram başarıyla kaydedildi 178686-0-0-17.png
Spectrogram başarıyla kaydedildi 178686-0-0-18.png
Spectrogram başarıyla kaydedildi 178686-0-0-19.png
Spectrogram başarıyla kaydedildi 178686-0-0-2.png
Spectrogram başarıyla kaydedildi 178686-0-0-21.png
Spectrogram başarıyla kaydedildi 178686-0-0-23.png
Spectrogram başarıyla kaydedildi 178686-0-0-3.png
Spectrogram başarıyla kaydedildi 178686-0-0-31.png
Spectrogram başarıyla kaydedildi 178686-0-0-33.png
Spectrogram başarıyla kaydedildi 178686-0-0-34.png
Spectrogram başarıyla kaydedildi 178686-0-0-38.png
Spectrogram başarıyla kaydedildi 178686-0-0-39.png
Spectrogram başarıyla kaydedildi 178686-0-0-42.png
Spectrogram başarıyla kaydedildi 178686-0-0-43.png
Spectrogram başarıyla kaydedildi 178686-0-0-44.png
Spectrogram başarıyla kaydedildi 178686-0-0-45.png
Spectrogram başarıyla kaydedildi 

Spectrogram başarıyla kaydedildi 189991-0-0-7.png
Spectrogram başarıyla kaydedildi 195969-0-0-1.png
Spectrogram başarıyla kaydedildi 195969-0-0-10.png
Spectrogram başarıyla kaydedildi 195969-0-0-12.png
Spectrogram başarıyla kaydedildi 195969-0-0-13.png
Spectrogram başarıyla kaydedildi 195969-0-0-14.png
Spectrogram başarıyla kaydedildi 195969-0-0-15.png
Spectrogram başarıyla kaydedildi 195969-0-0-17.png
Spectrogram başarıyla kaydedildi 195969-0-0-18.png
Spectrogram başarıyla kaydedildi 195969-0-0-19.png
Spectrogram başarıyla kaydedildi 195969-0-0-2.png
Spectrogram başarıyla kaydedildi 195969-0-0-20.png
Spectrogram başarıyla kaydedildi 195969-0-0-21.png
Spectrogram başarıyla kaydedildi 195969-0-0-22.png
Spectrogram başarıyla kaydedildi 195969-0-0-24.png
Spectrogram başarıyla kaydedildi 195969-0-0-25.png
Spectrogram başarıyla kaydedildi 195969-0-0-26.png
Spectrogram başarıyla kaydedildi 195969-0-0-28.png
Spectrogram başarıyla kaydedildi 195969-0-0-4.png
Spectrogram başarıyla kaydedildi 19

...

Spectrogram başarıyla kaydedildi 155044-9-0-37.png
Spectrogram başarıyla kaydedildi 155044-9-0-38.png
Spectrogram başarıyla kaydedildi 155127-9-0-2.png
Spectrogram başarıyla kaydedildi 155127-9-1-2.png
Spectrogram başarıyla kaydedildi 155127-9-1-23.png
Spectrogram başarıyla kaydedildi 155127-9-1-24.png
Spectrogram başarıyla kaydedildi 155127-9-1-25.png
Spectrogram başarıyla kaydedildi 155127-9-1-27.png
Spectrogram başarıyla kaydedildi 155202-9-0-124.png
Spectrogram başarıyla kaydedildi 155202-9-0-126.png
Spectrogram başarıyla kaydedildi 155202-9-0-135.png
Spectrogram başarıyla kaydedildi 155202-9-0-38.png
Spectrogram başarıyla kaydedildi 155202-9-0-42.png
Spectrogram başarıyla kaydedildi 155202-9-0-6.png
Spectrogram başarıyla kaydedildi 155212-9-0-13.png
Spectrogram başarıyla kaydedildi 155212-9-1-14.png
Spectrogram başarıyla kaydedildi 155212-9-1-49.png
Spectrogram başarıyla kaydedildi 155212-9-1-75.png
Spectrogram başarıyla kaydedildi 155212-9-1-85.png
Spectrogram başarıyla kaydedild

Spectrogram başarıyla kaydedildi 172519-9-0-10.png
Spectrogram başarıyla kaydedildi 172519-9-0-13.png
Spectrogram başarıyla kaydedildi 172519-9-0-49.png
Spectrogram başarıyla kaydedildi 172519-9-0-53.png
Spectrogram başarıyla kaydedildi 172519-9-0-67.png
Spectrogram başarıyla kaydedildi 172519-9-0-8.png
Spectrogram başarıyla kaydedildi 173891-9-0-1.png
Spectrogram başarıyla kaydedildi 173891-9-0-10.png
Spectrogram başarıyla kaydedildi 173891-9-0-11.png
Spectrogram başarıyla kaydedildi 173891-9-0-17.png
Spectrogram başarıyla kaydedildi 173891-9-0-2.png
Spectrogram başarıyla kaydedildi 173891-9-0-7.png
Spectrogram başarıyla kaydedildi 178099-9-0-0.png
Spectrogram başarıyla kaydedildi 178099-9-0-1.png
Spectrogram başarıyla kaydedildi 178099-9-0-10.png
Spectrogram başarıyla kaydedildi 178099-9-0-12.png
Spectrogram başarıyla kaydedildi 178099-9-0-6.png
Spectrogram başarıyla kaydedildi 178099-9-0-7.png
Spectrogram başarıyla kaydedildi 179039-9-0-22.png
Spectrogram başarıyla kaydedildi 179039

Spectrogram başarıyla kaydedildi 21684-9-0-7.png
Spectrogram başarıyla kaydedildi 26270-9-0-19.png
Spectrogram başarıyla kaydedildi 26270-9-0-23.png
Spectrogram başarıyla kaydedildi 26270-9-0-3.png
Spectrogram başarıyla kaydedildi 26270-9-0-30.png
Spectrogram başarıyla kaydedildi 26270-9-0-32.png
Spectrogram başarıyla kaydedildi 26270-9-0-35.png
Spectrogram başarıyla kaydedildi 28385-9-0-32.png
Spectrogram başarıyla kaydedildi 28385-9-0-33.png
Spectrogram başarıyla kaydedildi 28385-9-0-44.png
Spectrogram başarıyla kaydedildi 28385-9-0-46.png
Spectrogram başarıyla kaydedildi 28385-9-0-57.png
Spectrogram başarıyla kaydedildi 28385-9-0-65.png
Spectrogram başarıyla kaydedildi 28385-9-0-9.png
Spectrogram başarıyla kaydedildi 31973-9-0-43.png
Spectrogram başarıyla kaydedildi 31973-9-0-51.png
Spectrogram başarıyla kaydedildi 31973-9-0-56.png
Spectrogram başarıyla kaydedildi 31973-9-0-57.png
Spectrogram başarıyla kaydedildi 31973-9-0-64.png
Spectrogram başarıyla kaydedildi 31973-9-0-71.png
Spe

Spectrogram başarıyla kaydedildi 66000-9-0-3.png
Spectrogram başarıyla kaydedildi 66000-9-0-6.png
Spectrogram başarıyla kaydedildi 66000-9-0-8.png
Spectrogram başarıyla kaydedildi 66324-9-0-19.png
Spectrogram başarıyla kaydedildi 66324-9-0-30.png
Spectrogram başarıyla kaydedildi 66324-9-0-4.png
Spectrogram başarıyla kaydedildi 66324-9-0-42.png
Spectrogram başarıyla kaydedildi 66324-9-0-53.png
Spectrogram başarıyla kaydedildi 66324-9-0-54.png
Spectrogram başarıyla kaydedildi 66599-9-0-11.png
Spectrogram başarıyla kaydedildi 66599-9-0-17.png
Spectrogram başarıyla kaydedildi 66599-9-1-11.png
Spectrogram başarıyla kaydedildi 66599-9-1-21.png
Spectrogram başarıyla kaydedildi 66599-9-1-23.png
Spectrogram başarıyla kaydedildi 66599-9-1-4.png
Spectrogram başarıyla kaydedildi 69304-9-0-0.png
Spectrogram başarıyla kaydedildi 69304-9-0-12.png
Spectrogram başarıyla kaydedildi 69304-9-0-14.png
Spectrogram başarıyla kaydedildi 69304-9-0-15.png
Spectrogram başarıyla kaydedildi 69304-9-0-6.png
Spectro

Spectrogram başarıyla kaydedildi 94182-9-0-20.png
Spectrogram başarıyla kaydedildi 94182-9-0-23.png
Spectrogram başarıyla kaydedildi 94182-9-0-25.png
Spectrogram başarıyla kaydedildi 94182-9-0-9.png
Spectrogram başarıyla kaydedildi 94631-9-0-12.png
Spectrogram başarıyla kaydedildi 94631-9-0-16.png
Spectrogram başarıyla kaydedildi 94631-9-1-16.png
Spectrogram başarıyla kaydedildi 94631-9-1-2.png
Spectrogram başarıyla kaydedildi 94631-9-1-25.png
Spectrogram başarıyla kaydedildi 94631-9-1-3.png
Spectrogram başarıyla kaydedildi 96169-9-1-10.png
Spectrogram başarıyla kaydedildi 96169-9-1-3.png
Spectrogram başarıyla kaydedildi 96169-9-1-49.png
Spectrogram başarıyla kaydedildi 96169-9-1-55.png
Spectrogram başarıyla kaydedildi 96169-9-1-61.png
Spectrogram başarıyla kaydedildi 96169-9-1-74.png
Spectrogram başarıyla kaydedildi 96475-9-0-0.png
Spectrogram başarıyla kaydedildi 96475-9-0-3.png
Spectrogram başarıyla kaydedildi 96475-9-0-4.png
Spectrogram başarıyla kaydedildi 96475-9-0-5.png
Spectrog

# Creating a Tagged Data List
Bundan sonrasında ise veriyi bölmek ve modele sunmaya hazır hale getirmek için tüm veriyi etiketleyip kaydedeceğim, böylece veri proje arkadaşlarımın bilgisayarlarında da direkt olarak okunabilir hale gelecektir.


In [51]:
air_conditioner_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Spectrograms")
car_horn_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Spectrograms")
childeren_playing_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_playing/Spectrograms")
dog_bark_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark/Spectrograms")
drilling_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling/Spectrograms")
engine_idling_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling/Spectrograms")
gun_shot_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot/Spectrograms")
jackhammer_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer/Spectrograms")
siren_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Spectrograms")
street_music_sounds = os.listdir("C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Spectrograms")
tagged_data = []
print(tagged_data)



[]


In [52]:
a = 1
for i in air_conditioner_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/air_conditioner/Spectrograms/{i}"))
    pretagged.append("air_conditioner")
    tagged_data.append(pretagged)
    print(f'kalan: {len(air_conditioner_sounds)- a}')
    a += 1
print(len(tagged_data))
print(tagged_data[1])

kalan: 999
kalan: 998
kalan: 997
kalan: 996
kalan: 995
kalan: 994
kalan: 993
kalan: 992
kalan: 991
kalan: 990
kalan: 989
kalan: 988
kalan: 987
kalan: 986
kalan: 985
kalan: 984
kalan: 983
kalan: 982
kalan: 981
kalan: 980
kalan: 979
kalan: 978
kalan: 977
kalan: 976
kalan: 975
kalan: 974
kalan: 973
kalan: 972
kalan: 971
kalan: 970
kalan: 969
kalan: 968
kalan: 967
kalan: 966
kalan: 965
kalan: 964
kalan: 963
kalan: 962
kalan: 961
kalan: 960
kalan: 959
kalan: 958
kalan: 957
kalan: 956
kalan: 955
kalan: 954
kalan: 953
kalan: 952
kalan: 951
kalan: 950
kalan: 949
kalan: 948
kalan: 947
kalan: 946
kalan: 945
kalan: 944
kalan: 943
kalan: 942
kalan: 941
kalan: 940
kalan: 939
kalan: 938
kalan: 937
kalan: 936
kalan: 935
kalan: 934
kalan: 933
kalan: 932
kalan: 931
kalan: 930
kalan: 929
kalan: 928
kalan: 927
kalan: 926
kalan: 925
kalan: 924
kalan: 923
kalan: 922
kalan: 921
kalan: 920
kalan: 919
kalan: 918
kalan: 917
kalan: 916
kalan: 915
kalan: 914
kalan: 913
kalan: 912
kalan: 911
kalan: 910
kalan: 909

kalan: 237
kalan: 236
kalan: 235
kalan: 234
kalan: 233
kalan: 232
kalan: 231
kalan: 230
kalan: 229
kalan: 228
kalan: 227
kalan: 226
kalan: 225
kalan: 224
kalan: 223
kalan: 222
kalan: 221
kalan: 220
kalan: 219
kalan: 218
kalan: 217
kalan: 216
kalan: 215
kalan: 214
kalan: 213
kalan: 212
kalan: 211
kalan: 210
kalan: 209
kalan: 208
kalan: 207
kalan: 206
kalan: 205
kalan: 204
kalan: 203
kalan: 202
kalan: 201
kalan: 200
kalan: 199
kalan: 198
kalan: 197
kalan: 196
kalan: 195
kalan: 194
kalan: 193
kalan: 192
kalan: 191
kalan: 190
kalan: 189
kalan: 188
kalan: 187
kalan: 186
kalan: 185
kalan: 184
kalan: 183
kalan: 182
kalan: 181
kalan: 180
kalan: 179
kalan: 178
kalan: 177
kalan: 176
kalan: 175
kalan: 174
kalan: 173
kalan: 172
kalan: 171
kalan: 170
kalan: 169
kalan: 168
kalan: 167
kalan: 166
kalan: 165
kalan: 164
kalan: 163
kalan: 162
kalan: 161
kalan: 160
kalan: 159
kalan: 158
kalan: 157
kalan: 156
kalan: 155
kalan: 154
kalan: 153
kalan: 152
kalan: 151
kalan: 150
kalan: 149
kalan: 148
kalan: 147

In [53]:
index = len(tagged_data) -1
for i in car_horn_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/car_horn/Spectrograms/{i}"))
    pretagged.append("car_horn")
    tagged_data.append(pretagged)
print(len(tagged_data))
print(tagged_data[index])

1429
[array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255,

In [54]:
len(tagged_data)

1429

In [56]:
index = len(tagged_data) -1

a = 1
for i in childeren_playing_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/children_playing/Spectrograms/{i}"))
    pretagged.append("childeren_playing")
    tagged_data.append(pretagged)
    print('kalan:' + f"{len(childeren_playing_sounds)- a}")
    a += 1
print(len(tagged_data))
print(tagged_data[index])

kalan:999
kalan:998
kalan:997
kalan:996
kalan:995
kalan:994
kalan:993
kalan:992
kalan:991
kalan:990
kalan:989
kalan:988
kalan:987
kalan:986
kalan:985
kalan:984
kalan:983
kalan:982
kalan:981
kalan:980
kalan:979
kalan:978
kalan:977
kalan:976
kalan:975
kalan:974
kalan:973
kalan:972
kalan:971
kalan:970
kalan:969
kalan:968
kalan:967
kalan:966
kalan:965
kalan:964
kalan:963
kalan:962
kalan:961
kalan:960
kalan:959
kalan:958
kalan:957
kalan:956
kalan:955
kalan:954
kalan:953
kalan:952
kalan:951
kalan:950
kalan:949
kalan:948
kalan:947
kalan:946
kalan:945
kalan:944
kalan:943
kalan:942
kalan:941
kalan:940
kalan:939
kalan:938
kalan:937
kalan:936
kalan:935
kalan:934
kalan:933
kalan:932
kalan:931
kalan:930
kalan:929
kalan:928
kalan:927
kalan:926
kalan:925
kalan:924
kalan:923
kalan:922
kalan:921
kalan:920
kalan:919
kalan:918
kalan:917
kalan:916
kalan:915
kalan:914
kalan:913
kalan:912
kalan:911
kalan:910
kalan:909
kalan:908
kalan:907
kalan:906
kalan:905
kalan:904
kalan:903
kalan:902
kalan:901
kalan:900


kalan:123
kalan:122
kalan:121
kalan:120
kalan:119
kalan:118
kalan:117
kalan:116
kalan:115
kalan:114
kalan:113
kalan:112
kalan:111
kalan:110
kalan:109
kalan:108
kalan:107
kalan:106
kalan:105
kalan:104
kalan:103
kalan:102
kalan:101
kalan:100
kalan:99
kalan:98
kalan:97
kalan:96
kalan:95
kalan:94
kalan:93
kalan:92
kalan:91
kalan:90
kalan:89
kalan:88
kalan:87
kalan:86
kalan:85
kalan:84
kalan:83
kalan:82
kalan:81
kalan:80
kalan:79
kalan:78
kalan:77
kalan:76
kalan:75
kalan:74
kalan:73
kalan:72
kalan:71
kalan:70
kalan:69
kalan:68
kalan:67
kalan:66
kalan:65
kalan:64
kalan:63
kalan:62
kalan:61
kalan:60
kalan:59
kalan:58
kalan:57
kalan:56
kalan:55
kalan:54
kalan:53
kalan:52
kalan:51
kalan:50
kalan:49
kalan:48
kalan:47
kalan:46
kalan:45
kalan:44
kalan:43
kalan:42
kalan:41
kalan:40
kalan:39
kalan:38
kalan:37
kalan:36
kalan:35
kalan:34
kalan:33
kalan:32
kalan:31
kalan:30
kalan:29
kalan:28
kalan:27
kalan:26
kalan:25
kalan:24
kalan:23
kalan:22
kalan:21
kalan:20
kalan:19
kalan:18
kalan:17
kalan:16
kala

In [57]:
index = len(tagged_data) -1

a = 1
for i in dog_bark_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/dog_bark/Spectrograms/{i}"))
    pretagged.append("dog_bark")
    tagged_data.append(pretagged)
    print('kalan:' + f"{len(dog_bark_sounds)- a}")
    a += 1
print(len(tagged_data))
print(tagged_data[index])

kalan:999
kalan:998
kalan:997
kalan:996
kalan:995
kalan:994
kalan:993
kalan:992
kalan:991
kalan:990
kalan:989
kalan:988
kalan:987
kalan:986
kalan:985
kalan:984
kalan:983
kalan:982
kalan:981
kalan:980
kalan:979
kalan:978
kalan:977
kalan:976
kalan:975
kalan:974
kalan:973
kalan:972
kalan:971
kalan:970
kalan:969
kalan:968
kalan:967
kalan:966
kalan:965
kalan:964
kalan:963
kalan:962
kalan:961
kalan:960
kalan:959
kalan:958
kalan:957
kalan:956
kalan:955
kalan:954
kalan:953
kalan:952
kalan:951
kalan:950
kalan:949
kalan:948
kalan:947
kalan:946
kalan:945
kalan:944
kalan:943
kalan:942
kalan:941
kalan:940
kalan:939
kalan:938
kalan:937
kalan:936
kalan:935
kalan:934
kalan:933
kalan:932
kalan:931
kalan:930
kalan:929
kalan:928
kalan:927
kalan:926
kalan:925
kalan:924
kalan:923
kalan:922
kalan:921
kalan:920
kalan:919
kalan:918
kalan:917
kalan:916
kalan:915
kalan:914
kalan:913
kalan:912
kalan:911
kalan:910
kalan:909
kalan:908
kalan:907
kalan:906
kalan:905
kalan:904
kalan:903
kalan:902
kalan:901
kalan:900


kalan:179
kalan:178
kalan:177
kalan:176
kalan:175
kalan:174
kalan:173
kalan:172
kalan:171
kalan:170
kalan:169
kalan:168
kalan:167
kalan:166
kalan:165
kalan:164
kalan:163
kalan:162
kalan:161
kalan:160
kalan:159
kalan:158
kalan:157
kalan:156
kalan:155
kalan:154
kalan:153
kalan:152
kalan:151
kalan:150
kalan:149
kalan:148
kalan:147
kalan:146
kalan:145
kalan:144
kalan:143
kalan:142
kalan:141
kalan:140
kalan:139
kalan:138
kalan:137
kalan:136
kalan:135
kalan:134
kalan:133
kalan:132
kalan:131
kalan:130
kalan:129
kalan:128
kalan:127
kalan:126
kalan:125
kalan:124
kalan:123
kalan:122
kalan:121
kalan:120
kalan:119
kalan:118
kalan:117
kalan:116
kalan:115
kalan:114
kalan:113
kalan:112
kalan:111
kalan:110
kalan:109
kalan:108
kalan:107
kalan:106
kalan:105
kalan:104
kalan:103
kalan:102
kalan:101
kalan:100
kalan:99
kalan:98
kalan:97
kalan:96
kalan:95
kalan:94
kalan:93
kalan:92
kalan:91
kalan:90
kalan:89
kalan:88
kalan:87
kalan:86
kalan:85
kalan:84
kalan:83
kalan:82
kalan:81
kalan:80
kalan:79
kalan:78
ka

In [58]:
index = len(tagged_data) -1

a = 1

for i in drilling_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/drilling/Spectrograms/{i}"))
    pretagged.append("drilling")
    tagged_data.append(pretagged)
    print('kalan:' + f"{len(drilling_sounds)- a}")
    a += 1

print(len(tagged_data))
print(tagged_data[index])

kalan:999
kalan:998
kalan:997
kalan:996
kalan:995
kalan:994
kalan:993
kalan:992
kalan:991
kalan:990
kalan:989
kalan:988
kalan:987
kalan:986
kalan:985
kalan:984
kalan:983
kalan:982
kalan:981
kalan:980
kalan:979
kalan:978
kalan:977
kalan:976
kalan:975
kalan:974
kalan:973
kalan:972
kalan:971
kalan:970
kalan:969
kalan:968
kalan:967
kalan:966
kalan:965
kalan:964
kalan:963
kalan:962
kalan:961
kalan:960
kalan:959
kalan:958
kalan:957
kalan:956
kalan:955
kalan:954
kalan:953
kalan:952
kalan:951
kalan:950
kalan:949
kalan:948
kalan:947
kalan:946
kalan:945
kalan:944
kalan:943
kalan:942
kalan:941
kalan:940
kalan:939
kalan:938
kalan:937
kalan:936
kalan:935
kalan:934
kalan:933
kalan:932
kalan:931
kalan:930
kalan:929
kalan:928
kalan:927
kalan:926
kalan:925
kalan:924
kalan:923
kalan:922
kalan:921
kalan:920
kalan:919
kalan:918
kalan:917
kalan:916
kalan:915
kalan:914
kalan:913
kalan:912
kalan:911
kalan:910
kalan:909
kalan:908
kalan:907
kalan:906
kalan:905
kalan:904
kalan:903
kalan:902
kalan:901
kalan:900


kalan:169
kalan:168
kalan:167
kalan:166
kalan:165
kalan:164
kalan:163
kalan:162
kalan:161
kalan:160
kalan:159
kalan:158
kalan:157
kalan:156
kalan:155
kalan:154
kalan:153
kalan:152
kalan:151
kalan:150
kalan:149
kalan:148
kalan:147
kalan:146
kalan:145
kalan:144
kalan:143
kalan:142
kalan:141
kalan:140
kalan:139
kalan:138
kalan:137
kalan:136
kalan:135
kalan:134
kalan:133
kalan:132
kalan:131
kalan:130
kalan:129
kalan:128
kalan:127
kalan:126
kalan:125
kalan:124
kalan:123
kalan:122
kalan:121
kalan:120
kalan:119
kalan:118
kalan:117
kalan:116
kalan:115
kalan:114
kalan:113
kalan:112
kalan:111
kalan:110
kalan:109
kalan:108
kalan:107
kalan:106
kalan:105
kalan:104
kalan:103
kalan:102
kalan:101
kalan:100
kalan:99
kalan:98
kalan:97
kalan:96
kalan:95
kalan:94
kalan:93
kalan:92
kalan:91
kalan:90
kalan:89
kalan:88
kalan:87
kalan:86
kalan:85
kalan:84
kalan:83
kalan:82
kalan:81
kalan:80
kalan:79
kalan:78
kalan:77
kalan:76
kalan:75
kalan:74
kalan:73
kalan:72
kalan:71
kalan:70
kalan:69
kalan:68
kalan:67
kal

In [59]:
index = len(tagged_data) -1

a = 1
for i in engine_idling_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/engine_idling/Spectrograms/{i}"))
    pretagged.append("engine_idling")
    tagged_data.append(pretagged)
    print('kalan:' + f"{len(engine_idling_sounds)- a}")
    a += 1
print(len(tagged_data))
print(tagged_data[index])

kalan:999
kalan:998
kalan:997
kalan:996
kalan:995
kalan:994
kalan:993
kalan:992
kalan:991
kalan:990
kalan:989
kalan:988
kalan:987
kalan:986
kalan:985
kalan:984
kalan:983
kalan:982
kalan:981
kalan:980
kalan:979
kalan:978
kalan:977
kalan:976
kalan:975
kalan:974
kalan:973
kalan:972
kalan:971
kalan:970
kalan:969
kalan:968
kalan:967
kalan:966
kalan:965
kalan:964
kalan:963
kalan:962
kalan:961
kalan:960
kalan:959
kalan:958
kalan:957
kalan:956
kalan:955
kalan:954
kalan:953
kalan:952
kalan:951
kalan:950
kalan:949
kalan:948
kalan:947
kalan:946
kalan:945
kalan:944
kalan:943
kalan:942
kalan:941
kalan:940
kalan:939
kalan:938
kalan:937
kalan:936
kalan:935
kalan:934
kalan:933
kalan:932
kalan:931
kalan:930
kalan:929
kalan:928
kalan:927
kalan:926
kalan:925
kalan:924
kalan:923
kalan:922
kalan:921
kalan:920
kalan:919
kalan:918
kalan:917
kalan:916
kalan:915
kalan:914
kalan:913
kalan:912
kalan:911
kalan:910
kalan:909
kalan:908
kalan:907
kalan:906
kalan:905
kalan:904
kalan:903
kalan:902
kalan:901
kalan:900


kalan:165
kalan:164
kalan:163
kalan:162
kalan:161
kalan:160
kalan:159
kalan:158
kalan:157
kalan:156
kalan:155
kalan:154
kalan:153
kalan:152
kalan:151
kalan:150
kalan:149
kalan:148
kalan:147
kalan:146
kalan:145
kalan:144
kalan:143
kalan:142
kalan:141
kalan:140
kalan:139
kalan:138
kalan:137
kalan:136
kalan:135
kalan:134
kalan:133
kalan:132
kalan:131
kalan:130
kalan:129
kalan:128
kalan:127
kalan:126
kalan:125
kalan:124
kalan:123
kalan:122
kalan:121
kalan:120
kalan:119
kalan:118
kalan:117
kalan:116
kalan:115
kalan:114
kalan:113
kalan:112
kalan:111
kalan:110
kalan:109
kalan:108
kalan:107
kalan:106
kalan:105
kalan:104
kalan:103
kalan:102
kalan:101
kalan:100
kalan:99
kalan:98
kalan:97
kalan:96
kalan:95
kalan:94
kalan:93
kalan:92
kalan:91
kalan:90
kalan:89
kalan:88
kalan:87
kalan:86
kalan:85
kalan:84
kalan:83
kalan:82
kalan:81
kalan:80
kalan:79
kalan:78
kalan:77
kalan:76
kalan:75
kalan:74
kalan:73
kalan:72
kalan:71
kalan:70
kalan:69
kalan:68
kalan:67
kalan:66
kalan:65
kalan:64
kalan:63
kalan:6

In [60]:
index = len(tagged_data) -1

a = 1
for i in gun_shot_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/gun_shot/Spectrograms/{i}"))
    pretagged.append("gun_shot")
    tagged_data.append(pretagged)
    print('kalan:' + f"{len(gun_shot_sounds)- a}")
    a += 1
print(len(tagged_data))
print(tagged_data[index])

kalan:373
kalan:372
kalan:371
kalan:370
kalan:369
kalan:368
kalan:367
kalan:366
kalan:365
kalan:364
kalan:363
kalan:362
kalan:361
kalan:360
kalan:359
kalan:358
kalan:357
kalan:356
kalan:355
kalan:354
kalan:353
kalan:352
kalan:351
kalan:350
kalan:349
kalan:348
kalan:347
kalan:346
kalan:345
kalan:344
kalan:343
kalan:342
kalan:341
kalan:340
kalan:339
kalan:338
kalan:337
kalan:336
kalan:335
kalan:334
kalan:333
kalan:332
kalan:331
kalan:330
kalan:329
kalan:328
kalan:327
kalan:326
kalan:325
kalan:324
kalan:323
kalan:322
kalan:321
kalan:320
kalan:319
kalan:318
kalan:317
kalan:316
kalan:315
kalan:314
kalan:313
kalan:312
kalan:311
kalan:310
kalan:309
kalan:308
kalan:307
kalan:306
kalan:305
kalan:304
kalan:303
kalan:302
kalan:301
kalan:300
kalan:299
kalan:298
kalan:297
kalan:296
kalan:295
kalan:294
kalan:293
kalan:292
kalan:291
kalan:290
kalan:289
kalan:288
kalan:287
kalan:286
kalan:285
kalan:284
kalan:283
kalan:282
kalan:281
kalan:280
kalan:279
kalan:278
kalan:277
kalan:276
kalan:275
kalan:274


In [61]:
index = len(tagged_data) -1

a = 1
for i in jackhammer_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/jackhammer/Spectrograms/{i}"))
    pretagged.append("jackhammer")
    tagged_data.append(pretagged)
    print('kalan:' + f"{len(jackhammer_sounds)- a}")
    a += 1
print(len(tagged_data))
print(tagged_data[index])

kalan:999
kalan:998
kalan:997
kalan:996
kalan:995
kalan:994
kalan:993
kalan:992
kalan:991
kalan:990
kalan:989
kalan:988
kalan:987
kalan:986
kalan:985
kalan:984
kalan:983
kalan:982
kalan:981
kalan:980
kalan:979
kalan:978
kalan:977
kalan:976
kalan:975
kalan:974
kalan:973
kalan:972
kalan:971
kalan:970
kalan:969
kalan:968
kalan:967
kalan:966
kalan:965
kalan:964
kalan:963
kalan:962
kalan:961
kalan:960
kalan:959
kalan:958
kalan:957
kalan:956
kalan:955
kalan:954
kalan:953
kalan:952
kalan:951
kalan:950
kalan:949
kalan:948
kalan:947
kalan:946
kalan:945
kalan:944
kalan:943
kalan:942
kalan:941
kalan:940
kalan:939
kalan:938
kalan:937
kalan:936
kalan:935
kalan:934
kalan:933
kalan:932
kalan:931
kalan:930
kalan:929
kalan:928
kalan:927
kalan:926
kalan:925
kalan:924
kalan:923
kalan:922
kalan:921
kalan:920
kalan:919
kalan:918
kalan:917
kalan:916
kalan:915
kalan:914
kalan:913
kalan:912
kalan:911
kalan:910
kalan:909
kalan:908
kalan:907
kalan:906
kalan:905
kalan:904
kalan:903
kalan:902
kalan:901
kalan:900


kalan:168
kalan:167
kalan:166
kalan:165
kalan:164
kalan:163
kalan:162
kalan:161
kalan:160
kalan:159
kalan:158
kalan:157
kalan:156
kalan:155
kalan:154
kalan:153
kalan:152
kalan:151
kalan:150
kalan:149
kalan:148
kalan:147
kalan:146
kalan:145
kalan:144
kalan:143
kalan:142
kalan:141
kalan:140
kalan:139
kalan:138
kalan:137
kalan:136
kalan:135
kalan:134
kalan:133
kalan:132
kalan:131
kalan:130
kalan:129
kalan:128
kalan:127
kalan:126
kalan:125
kalan:124
kalan:123
kalan:122
kalan:121
kalan:120
kalan:119
kalan:118
kalan:117
kalan:116
kalan:115
kalan:114
kalan:113
kalan:112
kalan:111
kalan:110
kalan:109
kalan:108
kalan:107
kalan:106
kalan:105
kalan:104
kalan:103
kalan:102
kalan:101
kalan:100
kalan:99
kalan:98
kalan:97
kalan:96
kalan:95
kalan:94
kalan:93
kalan:92
kalan:91
kalan:90
kalan:89
kalan:88
kalan:87
kalan:86
kalan:85
kalan:84
kalan:83
kalan:82
kalan:81
kalan:80
kalan:79
kalan:78
kalan:77
kalan:76
kalan:75
kalan:74
kalan:73
kalan:72
kalan:71
kalan:70
kalan:69
kalan:68
kalan:67
kalan:66
kala

In [62]:
index = len(tagged_data) -1

a = 1
for i in siren_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/siren/Spectrograms/{i}"))
    pretagged.append("siren")
    tagged_data.append(pretagged)
    print('kalan:' + f"{len(siren_sounds)- a}")
    a += 1
print(len(tagged_data))
print(tagged_data[index])

kalan:928
kalan:927
kalan:926
kalan:925
kalan:924
kalan:923
kalan:922
kalan:921
kalan:920
kalan:919
kalan:918
kalan:917
kalan:916
kalan:915
kalan:914
kalan:913
kalan:912
kalan:911
kalan:910
kalan:909
kalan:908
kalan:907
kalan:906
kalan:905
kalan:904
kalan:903
kalan:902
kalan:901
kalan:900
kalan:899
kalan:898
kalan:897
kalan:896
kalan:895
kalan:894
kalan:893
kalan:892
kalan:891
kalan:890
kalan:889
kalan:888
kalan:887
kalan:886
kalan:885
kalan:884
kalan:883
kalan:882
kalan:881
kalan:880
kalan:879
kalan:878
kalan:877
kalan:876
kalan:875
kalan:874
kalan:873
kalan:872
kalan:871
kalan:870
kalan:869
kalan:868
kalan:867
kalan:866
kalan:865
kalan:864
kalan:863
kalan:862
kalan:861
kalan:860
kalan:859
kalan:858
kalan:857
kalan:856
kalan:855
kalan:854
kalan:853
kalan:852
kalan:851
kalan:850
kalan:849
kalan:848
kalan:847
kalan:846
kalan:845
kalan:844
kalan:843
kalan:842
kalan:841
kalan:840
kalan:839
kalan:838
kalan:837
kalan:836
kalan:835
kalan:834
kalan:833
kalan:832
kalan:831
kalan:830
kalan:829


kalan:54
kalan:53
kalan:52
kalan:51
kalan:50
kalan:49
kalan:48
kalan:47
kalan:46
kalan:45
kalan:44
kalan:43
kalan:42
kalan:41
kalan:40
kalan:39
kalan:38
kalan:37
kalan:36
kalan:35
kalan:34
kalan:33
kalan:32
kalan:31
kalan:30
kalan:29
kalan:28
kalan:27
kalan:26
kalan:25
kalan:24
kalan:23
kalan:22
kalan:21
kalan:20
kalan:19
kalan:18
kalan:17
kalan:16
kalan:15
kalan:14
kalan:13
kalan:12
kalan:11
kalan:10
kalan:9
kalan:8
kalan:7
kalan:6
kalan:5
kalan:4
kalan:3
kalan:2
kalan:1
kalan:0
7732
[array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

 

In [63]:
index = len(tagged_data) -1

a = 1
for i in street_music_sounds:
    pretagged = []
    pretagged.append(cv2.imread(f"C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data/street_music/Spectrograms/{i}"))
    pretagged.append("street_music")
    tagged_data.append(pretagged)
    print('kalan:' + f"{len(childeren_playing_sounds)- a}")
    a += 1

print(len(tagged_data))
print(tagged_data[index])

kalan:999
kalan:998
kalan:997
kalan:996
kalan:995
kalan:994
kalan:993
kalan:992
kalan:991
kalan:990
kalan:989
kalan:988
kalan:987
kalan:986
kalan:985
kalan:984
kalan:983
kalan:982
kalan:981
kalan:980
kalan:979
kalan:978
kalan:977
kalan:976
kalan:975
kalan:974
kalan:973
kalan:972
kalan:971
kalan:970
kalan:969
kalan:968
kalan:967
kalan:966
kalan:965
kalan:964
kalan:963
kalan:962
kalan:961
kalan:960
kalan:959
kalan:958
kalan:957
kalan:956
kalan:955
kalan:954
kalan:953
kalan:952
kalan:951
kalan:950
kalan:949
kalan:948
kalan:947
kalan:946
kalan:945
kalan:944
kalan:943
kalan:942
kalan:941
kalan:940
kalan:939
kalan:938
kalan:937
kalan:936
kalan:935
kalan:934
kalan:933
kalan:932
kalan:931
kalan:930
kalan:929
kalan:928
kalan:927
kalan:926
kalan:925
kalan:924
kalan:923
kalan:922
kalan:921
kalan:920
kalan:919
kalan:918
kalan:917
kalan:916
kalan:915
kalan:914
kalan:913
kalan:912
kalan:911
kalan:910
kalan:909
kalan:908
kalan:907
kalan:906
kalan:905
kalan:904
kalan:903
kalan:902
kalan:901
kalan:900


kalan:152
kalan:151
kalan:150
kalan:149
kalan:148
kalan:147
kalan:146
kalan:145
kalan:144
kalan:143
kalan:142
kalan:141
kalan:140
kalan:139
kalan:138
kalan:137
kalan:136
kalan:135
kalan:134
kalan:133
kalan:132
kalan:131
kalan:130
kalan:129
kalan:128
kalan:127
kalan:126
kalan:125
kalan:124
kalan:123
kalan:122
kalan:121
kalan:120
kalan:119
kalan:118
kalan:117
kalan:116
kalan:115
kalan:114
kalan:113
kalan:112
kalan:111
kalan:110
kalan:109
kalan:108
kalan:107
kalan:106
kalan:105
kalan:104
kalan:103
kalan:102
kalan:101
kalan:100
kalan:99
kalan:98
kalan:97
kalan:96
kalan:95
kalan:94
kalan:93
kalan:92
kalan:91
kalan:90
kalan:89
kalan:88
kalan:87
kalan:86
kalan:85
kalan:84
kalan:83
kalan:82
kalan:81
kalan:80
kalan:79
kalan:78
kalan:77
kalan:76
kalan:75
kalan:74
kalan:73
kalan:72
kalan:71
kalan:70
kalan:69
kalan:68
kalan:67
kalan:66
kalan:65
kalan:64
kalan:63
kalan:62
kalan:61
kalan:60
kalan:59
kalan:58
kalan:57
kalan:56
kalan:55
kalan:54
kalan:53
kalan:52
kalan:51
kalan:50
kalan:49
kalan:48
ka

In [64]:
print(len(tagged_data))

8732


In [65]:
#verimizin doğru çevrilip çevrilmediğine bakmak için içinden rastgele indekslere bakalım
print(tagged_data[3])
print(tagged_data[2000])


[array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]

Bu isimleri başka bilgisayarda da sürekli kullanılabilir hale getirmek için bir csv dosyasına yazmak istiyorum böylece kullanacak diğer bilgisayarlar bunu hemen alabilir ve üzerinde çalışmaya başlayabilir. Bunun yanında tabii ki de tüm arrayleri txt dosyalarına yazdırdım.


In [67]:

#şimdi bu verilerin hepsinin ilk indexindeki arrayleri birer txt dosyasına yazdıracağım böylece tekrar tekrar kullanılabilir olacaklardır
a = 0
sayac = 1
#bu verilerin karışmaması için isimlerini image_array_[numara]_[classID].txt şeklinde yapmak istiyorum ve bunu bir sözlük yapısı ile yapacağım
label_names = ["air_conditioner", "car_horn","childeren_playing", "dog_bark", "drilling", "engine_idling", "gun_shot", "jackhammer",
                  "siren", "street_music"
                  ]
indexes = [0,1,2,3,4,5,6,7,8,9]
index_dict = dict(zip(label_names, indexes))

for i, j in tagged_data:
    i_reshaped = i.reshape(i.shape[0], -1)
    np.savetxt(f'C:/Users/FIRAT/Desktop/UrbanSound8K/Processed_Data_Only_Spectrograms/image_array_{a}_{index_dict[j]}.txt',i_reshaped)
    
    a += 1
    print(f'image_array_{a}.txt başarıyla kaydedildi. Kalan: {len(tagged_data)-sayac}')
    sayac += 1


image_array_1.txt başarıyla kaydedildi. Kalan: 8731
image_array_2.txt başarıyla kaydedildi. Kalan: 8730
image_array_3.txt başarıyla kaydedildi. Kalan: 8729
image_array_4.txt başarıyla kaydedildi. Kalan: 8728
image_array_5.txt başarıyla kaydedildi. Kalan: 8727
image_array_6.txt başarıyla kaydedildi. Kalan: 8726
image_array_7.txt başarıyla kaydedildi. Kalan: 8725
image_array_8.txt başarıyla kaydedildi. Kalan: 8724
image_array_9.txt başarıyla kaydedildi. Kalan: 8723
image_array_10.txt başarıyla kaydedildi. Kalan: 8722
image_array_11.txt başarıyla kaydedildi. Kalan: 8721
image_array_12.txt başarıyla kaydedildi. Kalan: 8720
image_array_13.txt başarıyla kaydedildi. Kalan: 8719
image_array_14.txt başarıyla kaydedildi. Kalan: 8718
image_array_15.txt başarıyla kaydedildi. Kalan: 8717
image_array_16.txt başarıyla kaydedildi. Kalan: 8716
image_array_17.txt başarıyla kaydedildi. Kalan: 8715
image_array_18.txt başarıyla kaydedildi. Kalan: 8714
image_array_19.txt başarıyla kaydedildi. Kalan: 8713
im

image_array_190.txt başarıyla kaydedildi. Kalan: 8542
image_array_191.txt başarıyla kaydedildi. Kalan: 8541
image_array_192.txt başarıyla kaydedildi. Kalan: 8540
image_array_193.txt başarıyla kaydedildi. Kalan: 8539
image_array_194.txt başarıyla kaydedildi. Kalan: 8538
image_array_195.txt başarıyla kaydedildi. Kalan: 8537
image_array_196.txt başarıyla kaydedildi. Kalan: 8536
image_array_197.txt başarıyla kaydedildi. Kalan: 8535
image_array_198.txt başarıyla kaydedildi. Kalan: 8534
image_array_199.txt başarıyla kaydedildi. Kalan: 8533
image_array_200.txt başarıyla kaydedildi. Kalan: 8532
image_array_201.txt başarıyla kaydedildi. Kalan: 8531
image_array_202.txt başarıyla kaydedildi. Kalan: 8530
image_array_203.txt başarıyla kaydedildi. Kalan: 8529
image_array_204.txt başarıyla kaydedildi. Kalan: 8528
image_array_205.txt başarıyla kaydedildi. Kalan: 8527
image_array_206.txt başarıyla kaydedildi. Kalan: 8526
image_array_207.txt başarıyla kaydedildi. Kalan: 8525
image_array_208.txt başarıyl

image_array_343.txt başarıyla kaydedildi. Kalan: 8389
image_array_344.txt başarıyla kaydedildi. Kalan: 8388
image_array_345.txt başarıyla kaydedildi. Kalan: 8387
image_array_346.txt başarıyla kaydedildi. Kalan: 8386
image_array_347.txt başarıyla kaydedildi. Kalan: 8385
image_array_348.txt başarıyla kaydedildi. Kalan: 8384
image_array_349.txt başarıyla kaydedildi. Kalan: 8383
image_array_350.txt başarıyla kaydedildi. Kalan: 8382
image_array_351.txt başarıyla kaydedildi. Kalan: 8381
image_array_352.txt başarıyla kaydedildi. Kalan: 8380
image_array_353.txt başarıyla kaydedildi. Kalan: 8379
image_array_354.txt başarıyla kaydedildi. Kalan: 8378
image_array_355.txt başarıyla kaydedildi. Kalan: 8377
image_array_356.txt başarıyla kaydedildi. Kalan: 8376
image_array_357.txt başarıyla kaydedildi. Kalan: 8375
image_array_358.txt başarıyla kaydedildi. Kalan: 8374
image_array_359.txt başarıyla kaydedildi. Kalan: 8373
image_array_360.txt başarıyla kaydedildi. Kalan: 8372
image_array_361.txt başarıyl

image_array_519.txt başarıyla kaydedildi. Kalan: 8213
image_array_520.txt başarıyla kaydedildi. Kalan: 8212
image_array_521.txt başarıyla kaydedildi. Kalan: 8211
image_array_522.txt başarıyla kaydedildi. Kalan: 8210
image_array_523.txt başarıyla kaydedildi. Kalan: 8209
image_array_524.txt başarıyla kaydedildi. Kalan: 8208
image_array_525.txt başarıyla kaydedildi. Kalan: 8207
image_array_526.txt başarıyla kaydedildi. Kalan: 8206
image_array_527.txt başarıyla kaydedildi. Kalan: 8205
image_array_528.txt başarıyla kaydedildi. Kalan: 8204
image_array_529.txt başarıyla kaydedildi. Kalan: 8203
image_array_530.txt başarıyla kaydedildi. Kalan: 8202
image_array_531.txt başarıyla kaydedildi. Kalan: 8201
image_array_532.txt başarıyla kaydedildi. Kalan: 8200
image_array_533.txt başarıyla kaydedildi. Kalan: 8199
image_array_534.txt başarıyla kaydedildi. Kalan: 8198
image_array_535.txt başarıyla kaydedildi. Kalan: 8197
image_array_536.txt başarıyla kaydedildi. Kalan: 8196
image_array_537.txt başarıyl

image_array_699.txt başarıyla kaydedildi. Kalan: 8033
image_array_700.txt başarıyla kaydedildi. Kalan: 8032
image_array_701.txt başarıyla kaydedildi. Kalan: 8031
image_array_702.txt başarıyla kaydedildi. Kalan: 8030
image_array_703.txt başarıyla kaydedildi. Kalan: 8029
image_array_704.txt başarıyla kaydedildi. Kalan: 8028
image_array_705.txt başarıyla kaydedildi. Kalan: 8027
image_array_706.txt başarıyla kaydedildi. Kalan: 8026
image_array_707.txt başarıyla kaydedildi. Kalan: 8025
image_array_708.txt başarıyla kaydedildi. Kalan: 8024
image_array_709.txt başarıyla kaydedildi. Kalan: 8023
image_array_710.txt başarıyla kaydedildi. Kalan: 8022
image_array_711.txt başarıyla kaydedildi. Kalan: 8021
image_array_712.txt başarıyla kaydedildi. Kalan: 8020
image_array_713.txt başarıyla kaydedildi. Kalan: 8019
image_array_714.txt başarıyla kaydedildi. Kalan: 8018
image_array_715.txt başarıyla kaydedildi. Kalan: 8017
image_array_716.txt başarıyla kaydedildi. Kalan: 8016
image_array_717.txt başarıyl

...

image_array_8003.txt başarıyla kaydedildi. Kalan: 729
image_array_8004.txt başarıyla kaydedildi. Kalan: 728
image_array_8005.txt başarıyla kaydedildi. Kalan: 727
image_array_8006.txt başarıyla kaydedildi. Kalan: 726
image_array_8007.txt başarıyla kaydedildi. Kalan: 725
image_array_8008.txt başarıyla kaydedildi. Kalan: 724
image_array_8009.txt başarıyla kaydedildi. Kalan: 723
image_array_8010.txt başarıyla kaydedildi. Kalan: 722
image_array_8011.txt başarıyla kaydedildi. Kalan: 721
image_array_8012.txt başarıyla kaydedildi. Kalan: 720
image_array_8013.txt başarıyla kaydedildi. Kalan: 719
image_array_8014.txt başarıyla kaydedildi. Kalan: 718
image_array_8015.txt başarıyla kaydedildi. Kalan: 717
image_array_8016.txt başarıyla kaydedildi. Kalan: 716
image_array_8017.txt başarıyla kaydedildi. Kalan: 715
image_array_8018.txt başarıyla kaydedildi. Kalan: 714
image_array_8019.txt başarıyla kaydedildi. Kalan: 713
image_array_8020.txt başarıyla kaydedildi. Kalan: 712
image_array_8021.txt başarıy

image_array_8156.txt başarıyla kaydedildi. Kalan: 576
image_array_8157.txt başarıyla kaydedildi. Kalan: 575
image_array_8158.txt başarıyla kaydedildi. Kalan: 574
image_array_8159.txt başarıyla kaydedildi. Kalan: 573
image_array_8160.txt başarıyla kaydedildi. Kalan: 572
image_array_8161.txt başarıyla kaydedildi. Kalan: 571
image_array_8162.txt başarıyla kaydedildi. Kalan: 570
image_array_8163.txt başarıyla kaydedildi. Kalan: 569
image_array_8164.txt başarıyla kaydedildi. Kalan: 568
image_array_8165.txt başarıyla kaydedildi. Kalan: 567
image_array_8166.txt başarıyla kaydedildi. Kalan: 566
image_array_8167.txt başarıyla kaydedildi. Kalan: 565
image_array_8168.txt başarıyla kaydedildi. Kalan: 564
image_array_8169.txt başarıyla kaydedildi. Kalan: 563
image_array_8170.txt başarıyla kaydedildi. Kalan: 562
image_array_8171.txt başarıyla kaydedildi. Kalan: 561
image_array_8172.txt başarıyla kaydedildi. Kalan: 560
image_array_8173.txt başarıyla kaydedildi. Kalan: 559
image_array_8174.txt başarıy

image_array_8346.txt başarıyla kaydedildi. Kalan: 386
image_array_8347.txt başarıyla kaydedildi. Kalan: 385
image_array_8348.txt başarıyla kaydedildi. Kalan: 384
image_array_8349.txt başarıyla kaydedildi. Kalan: 383
image_array_8350.txt başarıyla kaydedildi. Kalan: 382
image_array_8351.txt başarıyla kaydedildi. Kalan: 381
image_array_8352.txt başarıyla kaydedildi. Kalan: 380
image_array_8353.txt başarıyla kaydedildi. Kalan: 379
image_array_8354.txt başarıyla kaydedildi. Kalan: 378
image_array_8355.txt başarıyla kaydedildi. Kalan: 377
image_array_8356.txt başarıyla kaydedildi. Kalan: 376
image_array_8357.txt başarıyla kaydedildi. Kalan: 375
image_array_8358.txt başarıyla kaydedildi. Kalan: 374
image_array_8359.txt başarıyla kaydedildi. Kalan: 373
image_array_8360.txt başarıyla kaydedildi. Kalan: 372
image_array_8361.txt başarıyla kaydedildi. Kalan: 371
image_array_8362.txt başarıyla kaydedildi. Kalan: 370
image_array_8363.txt başarıyla kaydedildi. Kalan: 369
image_array_8364.txt başarıy

image_array_8531.txt başarıyla kaydedildi. Kalan: 201
image_array_8532.txt başarıyla kaydedildi. Kalan: 200
image_array_8533.txt başarıyla kaydedildi. Kalan: 199
image_array_8534.txt başarıyla kaydedildi. Kalan: 198
image_array_8535.txt başarıyla kaydedildi. Kalan: 197
image_array_8536.txt başarıyla kaydedildi. Kalan: 196
image_array_8537.txt başarıyla kaydedildi. Kalan: 195
image_array_8538.txt başarıyla kaydedildi. Kalan: 194
image_array_8539.txt başarıyla kaydedildi. Kalan: 193
image_array_8540.txt başarıyla kaydedildi. Kalan: 192
image_array_8541.txt başarıyla kaydedildi. Kalan: 191
image_array_8542.txt başarıyla kaydedildi. Kalan: 190
image_array_8543.txt başarıyla kaydedildi. Kalan: 189
image_array_8544.txt başarıyla kaydedildi. Kalan: 188
image_array_8545.txt başarıyla kaydedildi. Kalan: 187
image_array_8546.txt başarıyla kaydedildi. Kalan: 186
image_array_8547.txt başarıyla kaydedildi. Kalan: 185
image_array_8548.txt başarıyla kaydedildi. Kalan: 184
image_array_8549.txt başarıy

image_array_8716.txt başarıyla kaydedildi. Kalan: 16
image_array_8717.txt başarıyla kaydedildi. Kalan: 15
image_array_8718.txt başarıyla kaydedildi. Kalan: 14
image_array_8719.txt başarıyla kaydedildi. Kalan: 13
image_array_8720.txt başarıyla kaydedildi. Kalan: 12
image_array_8721.txt başarıyla kaydedildi. Kalan: 11
image_array_8722.txt başarıyla kaydedildi. Kalan: 10
image_array_8723.txt başarıyla kaydedildi. Kalan: 9
image_array_8724.txt başarıyla kaydedildi. Kalan: 8
image_array_8725.txt başarıyla kaydedildi. Kalan: 7
image_array_8726.txt başarıyla kaydedildi. Kalan: 6
image_array_8727.txt başarıyla kaydedildi. Kalan: 5
image_array_8728.txt başarıyla kaydedildi. Kalan: 4
image_array_8729.txt başarıyla kaydedildi. Kalan: 3
image_array_8730.txt başarıyla kaydedildi. Kalan: 2
image_array_8731.txt başarıyla kaydedildi. Kalan: 1
image_array_8732.txt başarıyla kaydedildi. Kalan: 0


Listenin birinci elementlerini birer txt dosyasına kaydettim ki böylece tekrar işlememiz gerekmesin fakat bunu yapabilmem için üç boyutlu arrayleri iki boyuta düşürmem gerekti bu da modeli oluşturduğumuz notebookta bunları alırken tekrar şekillendirmemiz gerektiği anlamına gelmekte.

In [69]:
#Şimdi ise labelları bir listeye atacağız sonrasında ise kaydedeceğiz ve bunları ayırma işini modeli oluşturduğumuz notebookta yapacağız.
file = open('C:/Users/FIRAT/Desktop/UrbanSound8K/labels.txt', 'w', encoding = 'UTF-8')
for i, j in tagged_data:
    file.write(j + '\n')
    

file.close()
    

Artık görüldüğü üzere tüm verileri ve etiketleri çalışma ortamımıza kaydettik. Böylece artık ön işleme notebook'unu kapatabilir ve modeli oluşturmaya koyulabiliriz.

# Fırat ÇETİN